In [ ]:
n!pip install tensorflow==1.15 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 503 kB 60.9 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
     |████████████████████████████████| 3.8 MB 43.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=e6a3c79b6aba6e6896d7cdb2edcf8f7788c7d9f2a272ae76877763c1b2f19d14
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting un

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/Poli/Magistrale/Computational_Statistics/SindyAutoencoders/examples/lorenz_trial/src/

/gdrive/MyDrive/Poli/Magistrale/Computational_Statistics/SindyAutoencoders/examples/lorenz_trial/src


In [ ]:
import sys
#sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from mytraining import train_network
import tensorflow as tf

# Generate data

In [ ]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)

# Set up model and training parameters

In [ ]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0    #lambda2
params['loss_weight_sindy_x'] = 1e-4  #lambda1
params['loss_weight_sindy_regularization'] = 1e-5 #lambdad3

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 1501
params['refinement_epochs'] = 501

# Run training experiments

In [ ]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



EXPERIMENT 0


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


TRAINING
Epoch 0
   training loss 0.02994350902736187, (0.02907707, 484.46396, 8.577515, 0.86872953)
   validation loss 0.03688862547278404, (0.035583824, 424.86203, 12.96116, 0.86872953)
decoder loss ratio: 0.187890, decoder SINDy loss  ratio: 0.996888
Epoch 100
   training loss 3.52539645973593e-05, (7.5073194e-06, 0.27415627, 0.05775036, 2.197161)
   validation loss 6.588644464500248e-05, (2.3907816e-05, 0.7354935, 0.20007019, 2.197161)
decoder loss ratio: 0.000126, decoder SINDy loss  ratio: 0.015388
Epoch 200
   training loss 3.3412296033930033e-05, (1.2287532e-05, 0.057024032, 0.016639482, 1.9460818)
   validation loss 4.060156788909808e-05, (1.6417662e-05, 0.37468973, 0.047230925, 1.9460818)
decoder loss ratio: 0.000087, decoder SINDy loss  ratio: 0.003633
Epoch 300
   training loss 1.9924233129131608e-05, (3.3508886e-06, 0.062345028, 0.0089389775, 1.5679448)
   validation loss 2.5161480152746662e-05, (6.827469e-06, 0.45288345, 0.026545636, 1.5679448)
decoder loss ratio: 0.00003

KeyboardInterrupt: ignored